In [3]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from hungarian import Hungarian

In [17]:
class BirdsGraphDataset(Dataset):
    def __init__(self):
        super().__init__(use_graphwave=False)
    
    def __len__(self):
        return 11787
    
    def __getitem__(self):
        i, j = 0, 0
        while i == j:
            i, j = np.random.randint(0, 11786, 2)
            
        with open(f'../embeddings/{i}', 'rb') as fp:
            H1 = torch.load(fp).squeeze().cpu().detach().numpy()
        with open(f'../edge_embeddings/{i}', 'rb') as fp:
            E1 = torch.load(fp).squeeze().cpu().detach().numpy()
        with open(f'../adjacency_matrices/{i}.npy', 'rb') as fp:
            A1 = np.load(fp)
        
        
        
        with open(f'../embeddings/{j}', 'rb') as fp:
            H2 = torch.load(fp).squeeze().cpu().detach().numpy()
        with open(f'../edge_embeddings/{j}', 'rb') as fp:
            E2 = torch.load(fp).squeeze().cpu().detach().numpy()
        with open(f'../adjacency_matrices/{j}.npy', 'rb') as fp:
            A2 = np.load(fp)
            
        if use_graphwave:
            with open(f'../embeddings_graphwave/{i}', 'rb') as fp:
                G1 = np.load(fp)
            with open(f'../embeddings_graphwave/{j}', 'rb') as fp:
                G2 = np.load(fp)
            H1 = torch.cat([torch.tensor(H1), torch.tensor(G1)], dim=-1)
            H2 = torch.cat([torch.tensor(H2), torch.tensor(G2)], dim=-1)
            
        visible1 = A1.sum(axis=1)
        visible2 = A2.sum(axis=1)
        
        mask = ((visible1 & visible2) != 0).astype(int)
        
        return torch.tensor(H1), torch.tensor(E1), torch.tensor(H2), torch.tensor(E2), torch.tensor(mask)
        

* Идея: подсчитать все видимые ноды, которые были сматчены неправильно (правильный матчинг -- 1 в 1, 2 в 2 итд), подсчитать все невидимые ноды, которые были сматчены неправильно (в видимые ноды). Вычесть из 15 сумму этих двух чисел, получим количество правильных матчей. Его надо вернуть. 
* Маска видимости нодов -- 1д массив с единицами на позициях видимых нодов, нулями на позициях невидимых.

In [24]:
def compute_num_correct(S, mask):
    threshs = np.linspace(0, 100, 100)
    
    bs = S.shape[0]
    
    Z = torch.zeros_like(S, requires_grad=False)
    
    
    for i in range(bs):
        hungarian = Hungarian(S[i], is_profit_matrix=True)
        hungarian.calculate()

        idx = torch.tensor(hungarian.get_results())

        Z_buf = torch.zeros(S.shape, requires_grad=False)
        Z_buf[idx[:,0],idx[:,1]] = 1.

        Z_buf = Z_buf.long() 
        
        Z[i] = Z_buf
    
    ### YOUR CODE HERE ###
    
    return num_correct
        

In [25]:
def run_epoch(model, iterator, optimizer, criterion, phase='train', epoch=0, writer=None):
    
    is_train = (phase == 'train')
    if is_train:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0

    # variables for calculating accuracy
    n_predicted = 0
    n_true_predicted = 0
    
    with torch.set_grad_enabled(is_train):
        for i, batch in enumerate(iterator):
            global_i = len(iterator) * epoch + i
            
            # unpack batch
            H1, E1, H2, E2, mask = batch
            
            S_pred = model(H1, E1, H2, E2)
            
            loss = criterion(S_pred, torch.diag(mask))
            accuracy = compute_accuravy(S_pred, mask)
            l = loss.item()
            
            if is_train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            ## calculate accuracy
            ## add logging
            
                
            epoch_loss += loss.item()


        return epoch_loss / len(iterator)  # accuracy